COPYRIGHT © 2018 Kiran Arun <kironni@gmail.com>

### Setup

In [19]:
# setup
!rm -r Reinforcement_Learning-101-challenge
!git clone https://github.com/KiranArun/Reinforcement_Learning-101-challenge.git 1>/dev/null
!bash Reinforcement_Learning-101-challenge/scripts/setup.sh

Cloning into 'Reinforcement_Learning-101-challenge'...
remote: Counting objects: 87, done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 87 (delta 47), reused 55 (delta 23), pack-reused 0
Unpacking objects: 100% (87/87), done.
mkdir: cannot create directory ‘/content/models/’: File exists


In [20]:
# DEFINITELY DO NOT EDIT THIS CELL

import os
# Start noVNC remote Xwindows in a browser with TurboVNC
!bash /content/Reinforcement_Learning-101-challenge/scripts/kill-novnc.sh >/dev/null
!sleep 2
# start novnc server and expose via ngrok NOT HTTPS
os.environ['PATH'] += "${PATH}:/opt/VirtualGL/bin:/opt/TurboVNC/bin"
# hack line below to get vnc to start as ipython subprocess
!timeout 5 /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup 2>/dev/null
get_ipython().system_raw('python /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup > /content/.vnc/stdout 2>&1 &')
# ngrok remote tunnel
get_ipython().system_raw('/content/ngrok http 5901 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print('XWindow Link:', json.load(sys.stdin)['tunnels'][0]['public_url'])"
# get Xwindows going (right click to get openbox menu)
get_ipython().system_raw('DISPLAY=:1 openbox &')
# one-time-password
!grep "one-time password:" /content/.vnc/stdout

XWindow Link: http://a81125d2.ngrok.io
Full control one-time password: 23164052


In [0]:
# DO NOT EDIT THIS CELL

# set up Xvfb and attach it's own vncserver and test with spinning gears
!pkill Xvfb   # clean-up
!pkill x11vnc # clean-up

get_ipython().system_raw('DISPLAY=:1 xterm &')
get_ipython().system_raw('/usr/bin/Xvfb :99 -screen 0 640x480x24 &')
get_ipython().system_raw('/usr/bin/x11vnc -rfbport 5902 -forever -display :99 &')
!sleep 2
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 glxgears &')

In [0]:
# open a terminal into the glxgears session using TightVNC
get_ipython().system_raw('DISPLAY=:99 xterm -e /bin/bash -l -c "pkill glxgears"')
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 xterm -geometry 105x35 &')

# RL Challenge

In [0]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import gym

import keras
import keras.layers as layers
from rl import agents,memory,policy

### Hyperparameters

In [0]:
# Hyperparams

# number of steps to keep in experience buffer
memory_limit = 400

# discount value
gamma = 0.99

# how much to update target graph
target_model_update = 1e-2

# learning rate
learning_rate = 1e-2

# number of steps to sample from buffer to train on
batch_size = 32

In [25]:
# DO NOT EDIT

# setting environment
env = gym.make('CartPole-v1')
num_actions = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


### Neural Network

In [26]:
keras.backend.clear_session()

# create neural net
nn = keras.models.Sequential()

# keep this as input layer
nn.add(layers.Flatten(input_shape=(1,) + env.observation_space.shape))

# keep this as output layer
nn.add(layers.Dense(num_actions))

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


### Optimizer

https://keras.io/optimizers/

In [0]:
# set optimizer
optimizer = keras.optimizers.SGD(lr=learning_rate)

### Memory

In [0]:
# DO NOT EDIT

# set experience buffer
memory = memory.SequentialMemory(limit=memory_limit,
                                    window_length=1)

### Policy

choose from: https://github.com/keras-rl/keras-rl/blob/master/rl/policy.py

[info on choosing](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf)

In [0]:
# set policy
policy = policy.BoltzmannQPolicy()

### Agent

choose from: https://github.com/keras-rl/keras-rl/tree/master/rl/agents

and use: https://github.com/keras-rl/keras-rl/tree/master/examples

In [0]:
# create agent
model = agents.dqn.DQNAgent(model=nn,
                            nb_actions=num_actions,
                            memory=memory,
                            gamma=gamma,
                            batch_size=batch_size,
                            nb_steps_warmup=100,
                            target_model_update=target_model_update,
                            policy=policy)

In [0]:
# DO NOT EDIT

# compile model
model.compile(optimizer,
            metrics=['mae'])

### Training

In [32]:
# DO NOT EDIT

# train
history = model.fit(env,
                  nb_steps=1000,
                  verbose=2)

Training for 1000 steps ...
  10/1000: episode: 1, duration: 0.051s, episode steps: 10, steps per second: 198, episode reward: 10.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.700 [0.000, 1.000], mean observation: -0.133 [-1.657, 0.968], loss: --, mean_absolute_error: --, mean_q: --
  61/1000: episode: 2, duration: 0.032s, episode steps: 51, steps per second: 1572, episode reward: 51.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.451 [0.000, 1.000], mean observation: -0.097 [-1.316, 1.350], loss: --, mean_absolute_error: --, mean_q: --
  86/1000: episode: 3, duration: 0.016s, episode steps: 25, steps per second: 1608, episode reward: 25.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.600 [0.000, 1.000], mean observation: 0.014 [-1.782, 1.393], loss: --, mean_absolute_error: --, mean_q: --
 101/1000: episode: 4, duration: 0.282s, episode steps: 15, steps per second: 53, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.400 [0.000, 1.000], 

### Testing

In [33]:
# DO NOT EDIT

# test
hist = model.test(env, nb_episodes=50, visualize=False)
print('mean:', np.mean(hist.history['episode_reward']))

Testing for 50 episodes ...
Episode 1: reward: 10.000, steps: 10
Episode 2: reward: 8.000, steps: 8
Episode 3: reward: 9.000, steps: 9
Episode 4: reward: 10.000, steps: 10
Episode 5: reward: 8.000, steps: 8
Episode 6: reward: 10.000, steps: 10
Episode 7: reward: 10.000, steps: 10
Episode 8: reward: 10.000, steps: 10
Episode 9: reward: 10.000, steps: 10
Episode 10: reward: 10.000, steps: 10
Episode 11: reward: 9.000, steps: 9
Episode 12: reward: 10.000, steps: 10
Episode 13: reward: 9.000, steps: 9
Episode 14: reward: 9.000, steps: 9
Episode 15: reward: 8.000, steps: 8
Episode 16: reward: 8.000, steps: 8
Episode 17: reward: 10.000, steps: 10
Episode 18: reward: 9.000, steps: 9
Episode 19: reward: 10.000, steps: 10
Episode 20: reward: 11.000, steps: 11
Episode 21: reward: 9.000, steps: 9
Episode 22: reward: 8.000, steps: 8
Episode 23: reward: 10.000, steps: 10
Episode 24: reward: 11.000, steps: 11
Episode 25: reward: 9.000, steps: 9
Episode 26: reward: 11.000, steps: 11
Episode 27: rewar

### Displaying test games

The below cell will save just the neural net with its weights.

To run the test with display:

1. Go to the XWindow link, and login if you haven't already
2. In the TightVNC terminal, run:

    - `python3 Reinforcement_Learning-101-challenge/display_cartpole.py <number_repeats>`
    - replace `<number_repeats>` with however many games you want it to play


In [0]:
# save model to file
nn.save('/content/models/nn.hdf5', overwrite=True)